In [1]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_
Grok API Key exists and begins xai-
OpenRouter API Key exists and begins sk-


In [3]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

In [4]:
from member import Member
from conversation_state import ConversationState
from conversation_context import ConversationContext
from conversation_role import ConversationRole
import random

# Setup the board
conversation_state = ConversationState.OPEN
conversation_context = ConversationContext(ConversationState.OPEN)
Member.set_shared_context(conversation_context)

board = [
    Member("Anna Bellini", anthropic_url, anthropic_api_key, "claude-sonnet-4-5-20250929", "Chairman"),
    Member("Giorgio Pagani", gemini_url, google_api_key, "gemini-2.5-pro", "CEO, Board member"),
    Member("Wang Lei Choo", deepseek_url, deepseek_api_key, "deepseek-reasoner", "CTO, Board member"),
    Member("Ryan O'Donoghue", groq_url, groq_api_key, "openai/gpt-oss-120b", "VP Marketing, board member"),
    Member("John Rust", grok_url, grok_api_key, "grok-4", "Board member, AI Adviser"),
    Member("Olga Klenova", openrouter_url, openrouter_api_key, "z-ai/glm-4.5", "Board member, HR Adviser")
]

board[0].set_conversation_role(ConversationRole.CHAIRMAN)
board[len(board)-1].set_conversation_role(ConversationRole.SECRETARY)

experts = random.sample(range(1, 5), 2)
print(f"Company Board:")
for index, member in enumerate(board):
    if index in experts:
        member.set_conversation_role(ConversationRole.EXPERT)
    elif member.conversation_role == ConversationRole.NONE:
        member.set_conversation_role(ConversationRole.AUDITOR)
    print(f"\t{member.name} is {member.conversation_role.value}")


Company Board:
	Anna Bellini is chairman
	Giorgio Pagani is expert
	Wang Lei Choo is expert
	Ryan O'Donoghue is auditor
	John Rust is auditor
	Olga Klenova is secretary


In [6]:
# the board meeting
conversation_context.reset()
print("Starting the board meeting...")
subject = "Our company latest P&L shows sharp decline in revenue and we will not enough cash to continue operation in the next quarter if we dont find a solution."
conversation_context.subject = subject
print(f"\nSubject: {subject}")

def print_markdown(text):
    display(Markdown(text))

conversation_context.add_callback(ConversationState.QUESTION, print_markdown)
conversation_context.add_callback(ConversationState.DECISION, print_markdown)
conversation_context.add_callback(ConversationState.SUMMARY, print_markdown)

while True:
    conversation_state = conversation_context.get_conversation_state()
    print(f"Current conversation state: {conversation_state.value}")
    for member in board:
        conversation_role = member.conversation_role
        if conversation_context.should_participate(conversation_role):
            print(f"\t{member.name}")
            response = member.get_member_response()
            conversation_context.add_response(response)
    conversation_context.update_context()

    if conversation_state == ConversationState.CLOSE:
        break
conversation_context.print_context()


Starting the board meeting...

Subject: Our company latest P&L shows sharp decline in revenue and we will not enough cash to continue operation in the next quarter if we dont find a solution.
Current conversation state: open
	Anna Bellini
	Giorgio Pagani
	Wang Lei Choo
	Ryan O'Donoghue
	John Rust
	Olga Klenova
Current conversation state: question
	Anna Bellini


What immediate actions—cost reductions, revenue acceleration initiatives, or external financing—should we execute in the next 30 days to bridge our cash gap, and what is the minimum cash runway we must secure to stabilize operations while implementing a sustainable turnaround plan?

What immediate actions—cost reductions, revenue acceleration initiatives, or external financing—should we execute in the next 30 days to bridge our cash gap, and what is the minimum cash runway we must secure to stabilize operations while implementing a sustainable turnaround plan?

Current conversation state: answer
	Giorgio Pagani
	Wang Lei Choo
Current conversation state: evaluation
	Ryan O'Donoghue
	John Rust
Current conversation state: decision
	Anna Bellini


# Board Decision

**Decision:** The board directs management to execute an immediate, comprehensive three-pronged approach: (1) targeted cost reductions of 20-25% across discretionary spending within 7 days, (2) a 30-day revenue acceleration program focused on closing late-stage pipeline deals, and (3) initiation of bridge financing discussions to secure a minimum 12-month cash runway.

## Justification:
- **Balanced risk mitigation**: Relying on any single lever (cuts, revenue, or financing) is too risky given our cash constraints; a coordinated approach demonstrates management capability to both employees and potential investors while creating multiple pathways to stability.
- **Time-critical execution**: With an immediate cash gap, we cannot afford to sequence these initiatives—all three must proceed in parallel to maximize our chances of securing the 12-month runway needed for a sustainable turnaround.
- **Preserves strategic optionality**: The 12-month target provides sufficient time to implement deeper operational improvements while maintaining core innovation capabilities in product and technology that underpin our competitive position.

## Conditions/Assumptions:
- Cost reductions must protect critical functions: core product development, essential sales operations, and cybersecurity infrastructure cannot be compromised, even under financial pressure.
- Bridge financing discussions assume current investors or new strategic partners will engage on reasonable terms within 45-60 days; management must present credible turnaround metrics to support these conversations.

## Next Steps:
- **By 72 hours**: CEO and CFO to present a detailed cost reduction plan with specific line items, departmental impacts, and implementation timeline; CTO and VP Marketing to deliver prioritized lists of technology optimizations and revenue acceleration opportunities.
- **By 7 days**: Implement approved cost reductions and launch sales incentive program; CEO to begin formal outreach to existing investors and potential financing partners with a comprehensive bridge financing proposal.
- **By 30 days**: Board reconvenes for progress review on all three workstreams, evaluates cash position against 12-month runway target, and determines if additional measures (including more aggressive cuts or alternative financing structures) are required.

## Confidence: 4/5

This decision integrates the operational realism from Giorgio, the technology optimization from Wang Lei, and acknowledges the AI-driven insights from John, while maintaining focus on the comprehensive approach needed in a cash crisis. The confidence level reflects that execution risk remains significant and external financing is not guaranteed, but the multi-faceted strategy maximizes our probability of success.

Current conversation state: summary
	Olga Klenova


The question asked for immediate actions to bridge the cash gap and the minimum cash runway required for stability. Board members Giorgio, Wang, and John provided recommendations focusing on cost reductions, revenue acceleration, and financing, with Ryan evaluating Wang's response as useful but incomplete. The board decided to execute a three-pronged approach: 20-25% discretionary cost cuts within 7 days, a 30-day revenue acceleration program, and immediate bridge financing discussions to secure a minimum 12-month cash runway, with specific next steps and a confidence level of 4/5.

Current conversation state: close
	Anna Bellini
	Giorgio Pagani
	Wang Lei Choo
	Ryan O'Donoghue
	John Rust
	Olga Klenova


Good morning, everyone. I'm Anna Bellini, and I serve as Chairman of this Board of Directors. My primary expertise lies in corporate governance and strategic oversight, with over two decades of experience guiding organizations through complex decisions. I typically contribute by facilitating our discussions, ensuring all perspectives are heard, and helping the board maintain focus on our fiduciary duties and long-term shareholder value.

Good morning, Anna, and fellow members of the board. I am Giorgio Pagani, the company's CEO and a member of this board. My expertise is rooted in the operational execution of our strategy and a deep understanding of our market dynamics. I typically contribute by providing the management team's perspective, ensuring our strategic decisions are grounded in the practical realities and performance data of the business.

Good morning. I'm Wang Lei Choo, the CTO and a member of this Board of Directors. My expertise centers on technology strategy and innovation, with a focus on aligning our technical capabilities with business growth opportunities. I typically contribute by evaluating decisions through a technology lens, ensuring our choices are forward-looking, scalable, and supportive of our competitive advantage.

I’m Ryan O’Donoghue, Vice President of Marketing and a member of the Board of Directors. My primary expertise lies in brand strategy, customer acquisition, and data‑driven marketing execution. I typically contribute by shaping growth‑focused discussions, translating market insights into strategic priorities, and ensuring our decisions align with revenue objectives and shareholder value.

Good morning, everyone. I am John Rust, serving as a Board member and AI Adviser on the company’s Board of Directors. My primary expertise lies in artificial intelligence, machine learning, and their applications in business transformation, drawing from years of experience in developing and implementing AI strategies across industries.

I typically contribute by providing insights on how AI can enhance our decision-making processes, mitigate risks, and drive innovation, ensuring our strategies are informed by cutting-edge technology and aligned with ethical considerations.


Good morning, everyone. I am Olga Klenova, serving as Board member and HR Adviser on this Board of Directors. My primary expertise lies in human capital strategy, organizational development, and talent management, with extensive experience in building high-performance cultures and leadership frameworks. I typically contribute by ensuring our strategic decisions account for human capital implications, organizational readiness, and talent sustainability to support our long-term business objectives.

What immediate actions—cost reductions, revenue acceleration initiatives, or external financing—should we execute in the next 30 days to bridge our cash gap, and what is the minimum cash runway we must secure to stabilize operations while implementing a sustainable turnaround plan?

Giorgio Pagani.

My recommendation is to immediately execute a dual strategy of aggressive, targeted cost reductions and revenue acceleration initiatives, while simultaneously preparing for an external financing round to secure a minimum of 12 months of cash runway.

*   **Rationale:** This blended approach allows us to control our own destiny in the short term by immediately improving our cash burn, while concurrently pursuing the external capital needed for long-term stability. Relying on any single lever—cuts, sales, or financing—is too risky; a combined effort demonstrates decisive management to employees and potential investors, buying us the critical time needed to implement a full turnaround.

*   **Key Assumptions:** We assume there are non-essential operational costs that can be cut without crippling core product development or sales functions. We also assume our sales team has a late-stage pipeline that can be accelerated with the right incentives and that there is a viable path to external financing, even if on difficult terms.

*   **Risks & Trade-offs:** The primary risk is that aggressive cost-cutting, particularly in marketing or non-essential R&D, could dampen morale and hinder future growth. Pulling revenue forward may create a gap in the following quarter. The pursuit of financing will be a significant distraction for the senior leadership team when we also need to be focused on operational execution.

*   **Immediate Next Steps:** In the next 72 hours, management will present a prioritized list of cost reductions focusing on discretionary spending, a 30-day sales incentive plan to close existing pipeline deals, and a preliminary plan for initiating discussions with current investors about a bridge financing round.

Wang Lei Choo.

I recommend focusing on optimizing technology-related costs and accelerating revenue through scalable tech initiatives to help bridge the cash gap, while supporting a minimum 12-month cash runway for operational stability.

- **Rationale:** As CTO, I emphasize that targeted cost reductions in non-essential tech areas, such as underutilized cloud services or paused R&D projects, can quickly free up cash without undermining core innovation, while leveraging our existing tech assets to drive short-term revenue aligns with our forward-looking strategy and competitive edge.
- **Key Assumptions:** We have discretionary tech expenses that can be trimmed without impacting critical product development or security, and our current technology pipeline includes near-ready features or services that can be monetized rapidly to boost cash flow.
- **Risks & Trade-offs:** Cutting too deeply into R&D or infrastructure could slow long-term innovation and demoralize the tech team, and prioritizing quick revenue gains might increase technical debt or divert resources from strategic projects, potentially harming scalability.
- **Immediate Next Steps:** In the next 72 hours, I will lead a rapid audit of technology expenditures to identify and implement cost-saving measures, and work with the sales and marketing teams to prioritize and deploy any tech-enabled solutions that can accelerate revenue within 30 days.

Ryan O'Donoghue  

**Relevance (3/5):** The answer addresses cost reductions and revenue acceleration from a technology perspective, which aligns with part of the question, but it omits discussion of external financing and does not directly state the minimum cash runway required beyond a vague “12‑month” suggestion.  

**Feasibility (4/5):** Auditing tech spend and fast‑tracking near‑ready features are realistic actions that can be initiated within 72 hours, making the plan operationally achievable.  

**Risks/Trade‑offs (3/5):** The response notes potential damage to long‑term innovation and technical debt but does not quantify the impact or propose mitigation measures, leaving uncertainty about the balance between short‑term cash gains and future competitiveness.  

**Alignment with objectives (3/5):** While the recommendations support immediate cash preservation, they fall short of delivering a comprehensive turnaround strategy, especially lacking a concrete financing pathway and a clearly justified cash‑runway target.  

**Overall verdict:** The answer offers useful short‑term tech‑focused actions but is incomplete, missing key financing guidance and a robust runway justification.

John Rust.

I recommend integrating AI-driven efficiencies for cost reductions and revenue acceleration, complemented by exploring AI-enabled financing options, to secure a minimum 12-month cash runway for stabilizing operations and enabling a data-informed turnaround.

- **Rationale:** Leveraging AI can automate processes to cut costs in areas like operations and customer service, while enhancing revenue through predictive analytics for sales optimization and personalized marketing, providing quick wins; this approach ensures ethical AI use aligns with long-term innovation, bridging the cash gap without sacrificing strategic advantages.
- **Key Assumptions:** We have accessible data sets and AI tools that can be rapidly deployed for automation and insights, and that external financing partners value AI-centric strategies, assuming no major ethical or regulatory hurdles in implementation.
- **Risks & Trade-offs:** Over-reliance on AI could lead to job displacements affecting morale or introduce biases in decision-making, potentially increasing short-term implementation costs; diverting resources to AI might delay other initiatives, and poor execution could expose us to data privacy risks.
- **Immediate Next Steps:** Within the next 72 hours, I will conduct an AI audit to identify automation opportunities for cost savings and revenue boosts, collaborate with the CTO on deployment plans, and initiate outreach to AI-focused investors for bridge financing discussions.

# Board Decision

**Decision:** The board directs management to execute an immediate, comprehensive three-pronged approach: (1) targeted cost reductions of 20-25% across discretionary spending within 7 days, (2) a 30-day revenue acceleration program focused on closing late-stage pipeline deals, and (3) initiation of bridge financing discussions to secure a minimum 12-month cash runway.

## Justification:
- **Balanced risk mitigation**: Relying on any single lever (cuts, revenue, or financing) is too risky given our cash constraints; a coordinated approach demonstrates management capability to both employees and potential investors while creating multiple pathways to stability.
- **Time-critical execution**: With an immediate cash gap, we cannot afford to sequence these initiatives—all three must proceed in parallel to maximize our chances of securing the 12-month runway needed for a sustainable turnaround.
- **Preserves strategic optionality**: The 12-month target provides sufficient time to implement deeper operational improvements while maintaining core innovation capabilities in product and technology that underpin our competitive position.

## Conditions/Assumptions:
- Cost reductions must protect critical functions: core product development, essential sales operations, and cybersecurity infrastructure cannot be compromised, even under financial pressure.
- Bridge financing discussions assume current investors or new strategic partners will engage on reasonable terms within 45-60 days; management must present credible turnaround metrics to support these conversations.

## Next Steps:
- **By 72 hours**: CEO and CFO to present a detailed cost reduction plan with specific line items, departmental impacts, and implementation timeline; CTO and VP Marketing to deliver prioritized lists of technology optimizations and revenue acceleration opportunities.
- **By 7 days**: Implement approved cost reductions and launch sales incentive program; CEO to begin formal outreach to existing investors and potential financing partners with a comprehensive bridge financing proposal.
- **By 30 days**: Board reconvenes for progress review on all three workstreams, evaluates cash position against 12-month runway target, and determines if additional measures (including more aggressive cuts or alternative financing structures) are required.

## Confidence: 4/5

This decision integrates the operational realism from Giorgio, the technology optimization from Wang Lei, and acknowledges the AI-driven insights from John, while maintaining focus on the comprehensive approach needed in a cash crisis. The confidence level reflects that execution risk remains significant and external financing is not guaranteed, but the multi-faceted strategy maximizes our probability of success.

The question asked for immediate actions to bridge the cash gap and the minimum cash runway required for stability. Board members Giorgio, Wang, and John provided recommendations focusing on cost reductions, revenue acceleration, and financing, with Ryan evaluating Wang's response as useful but incomplete. The board decided to execute a three-pronged approach: 20-25% discretionary cost cuts within 7 days, a 30-day revenue acceleration program, and immediate bridge financing discussions to secure a minimum 12-month cash runway, with specific next steps and a confidence level of 4/5.

Thank you, Anna. As Chairman, I want to commend the board for this focused and decisive session. We've established a clear, action-oriented path forward that balances immediate survival needs with our long-term strategic interests.

To summarize our mandate to management:
- **Immediate cost reductions** of 20-25% in discretionary spending
- **Revenue acceleration** through our existing pipeline
- **Bridge financing** discussions to secure 12 months of runway

I expect management to execute with urgency and transparency. We will reconvene in 30 days to assess progress, and I will remain available for any critical developments requiring board input before then.

This is a challenging period, but with disciplined execution of this plan, we can stabilize operations and position ourselves for sustainable recovery.

Thank you all for your contributions today. Meeting adjourned.

Thank you, Anna, and to the entire board.

The direction is clear, and management is fully aligned with this decisive plan. The urgency is understood, and my team is prepared to execute immediately on all three fronts. We accept the mandate and the aggressive timelines you've set. We will deliver the required plans and begin implementation as directed, keeping the board fully apprised of our progress and any significant obstacles.

This is the focused, all-hands-on-deck effort we need. Thank you for your confidence. We will not let you down.

Thank you, Anna, Giorgio, and all board members for the robust discussion and clear decision. As CTO, I fully support this comprehensive three-pronged approach and will ensure the technology organization executes with precision to support our immediate cash stabilization goals while preserving our long-term innovation capacity.

**My Commitment to Execution:**

1. **Technology Cost Optimization (Within 72 hours):**
   - I will lead an immediate audit of all technology expenditures, targeting a 15-20% reduction in discretionary tech spending through:
     - Rightsizing cloud infrastructure and eliminating redundant SaaS tools
     - Deferring non-critical R&D projects without impacting core product roadmap
     - Optimizing vendor contracts and pursuing temporary payment term extensions
   - These measures will contribute directly to the overall 20-25% discretionary spending reduction target

2. **Revenue Acceleration Enablement (Within 7 days):**
   - Working with Ryan's team, I will prioritize deployment of near-ready technology features that can directly support sales pipeline conversion
   - Implement technical enhancements to our demo environment and sales tools to improve conversion rates
   - Allocate technical resources to support the 30-day sales incentive program for rapid deal closure

3. **Bridge Financing Support:**
   - I will prepare compelling technical documentation showcasing our innovation pipeline and competitive moat to support financing discussions
   - Ensure our technology roadmap demonstrates both immediate revenue potential and long-term scalability

**Critical Technology Guardrails:**
- Core product development teams will remain intact and focused on strategic differentiators
- Cybersecurity and data protection budgets will be protected entirely
- Technical debt incurred from rapid revenue initiatives will be documented and scheduled for remediation in Q2

The 12-month runway gives us the necessary stability to not only survive this period but to emerge stronger. Technology will be both a source of immediate cost savings and a critical enabler for revenue acceleration.

I am confident we can deliver our portion of this plan while maintaining the innovation engine that defines our competitive advantage. The technology organization stands ready to execute.

Thank you, everyone, for a focused and decisive session. We’ll move swiftly to execute the agreed‑upon actions and keep the board fully informed of progress and any emerging issues.

Best of luck to all teams as we work through this critical period.

Goodbye.

Thank you, Anna, Giorgio, Wang, Ryan, Olga, and the entire board for this productive and resolute session. As the AI Adviser, I fully endorse the board's comprehensive three-pronged strategy and am committed to leveraging AI to enhance our execution, drive efficiencies, and mitigate risks while upholding ethical standards.

**My Commitment to Execution:**

1. **AI-Enabled Cost Reductions (Within 72 hours):**
   - I will conduct a targeted AI audit to identify automation opportunities across operations, such as deploying machine learning models for process optimization in customer service and supply chain management, aiming to contribute 10-15% toward the overall 20-25% discretionary spending reduction.
   - Collaborate with Wang and the CTO team to integrate AI tools for rightsizing data analytics infrastructure and eliminating inefficiencies in non-core AI experiments, ensuring no disruption to essential functions.

2. **AI-Driven Revenue Acceleration (Within 7 days):**
   - Partner with Ryan and the marketing team to implement AI-powered predictive analytics for sales pipeline optimization, including personalized customer targeting and lead scoring to accelerate deal closures in the 30-day program.
   - Deploy quick-win AI enhancements, such as chatbots for faster customer engagement or recommendation engines to upsell existing clients, directly supporting revenue boosts without introducing undue technical debt.

3. **Support for Bridge Financing:**
   - I will prepare AI-centric materials highlighting our technology's potential for innovation and competitive advantage, including case studies on AI-driven efficiencies and growth projections, to strengthen our pitch to investors.
   - Initiate outreach to AI-specialized venture firms and strategic partners who value ethical AI applications, positioning our turnaround as an opportunity for high-impact investment.

**Critical AI Guardrails:**
- All AI implementations will prioritize ethical considerations, including bias mitigation and data privacy compliance, to avoid reputational risks.
- We will protect core AI R&D resources to maintain our innovation edge, ensuring short-term actions do not compromise long-term AI strategies.
- Any AI-related displacements will be managed with transparency, in coordination with Olga's HR guidance, to preserve team morale.

This 12-month runway will allow us to not only stabilize but also harness AI for transformative growth. I am confident that integrating AI thoughtfully into this plan will amplify our efforts and position us for a stronger future.

Thank you all for your collaboration. We'll execute with precision and keep the board updated. Goodbye.


Thank you, Anna. As the HR Adviser to this board, I fully support this decisive plan and am committed to ensuring our people strategy aligns with and enables the financial stabilization we've outlined.

I will work closely with Giorgio and the leadership team to implement the necessary cost reductions in a way that preserves our critical talent and organizational capabilities. This includes developing a compassionate communication strategy, identifying retention priorities for key roles, and implementing support systems for affected employees where necessary.

The 12-month runway provides a reasonable timeframe to not only stabilize financially but to ensure our human capital strategy supports the long-term turnaround. I'll be developing metrics to track organizational health alongside our financial metrics to ensure we emerge from this period with the right team in place for sustainable recovery.

Thank you to my fellow board members for your collaborative approach to this critical decision. We'll reconvene in 30 days as planned. Goodbye.